In [ ]:
import os
import cv2
import numpy as np

In [ ]:
# load saliency and motion videos and split them into chunks
path = '../data/'

In [83]:
def extractImages(video):
    newdir = path+'images'
    if os.path.exists(newdir):
        os.system('rm -rf '+newdir)
    os.makedirs(newdir)
    os.system('ffmpeg -i '+path+video+' -vf fps=30 -s 3840x1920 '+newdir+'/frame%0000d.png')

In [84]:
video = path+'RollerCoaster1min.mp4'
extractImages(video)

In [178]:
def makeNumpyArrayInputs(totalImages):
    frame_no = 1
    for imageName in totalImages:
        windowsize_r = 192
        windowsize_c = 192
        tile_no = 1
        tiles = []
        image = path+'images/'+imageName
        image = cv2.imread(image)
        for r in range(0,image.shape[0], windowsize_r):
            for c in range(0,image.shape[1], windowsize_c):
                window = image[r:r+windowsize_r,c:c+windowsize_c]
                tsegs = []
                for x in range(windowsize_r):
                    temp = np.append(window[x], np.ones((192,1), dtype=np.uint8)*tile_no, axis=1)
                    temp2 = np.append(temp, np.ones((192,1), dtype=np.uint8)*frame_no, axis=1)
                    tsegs.append(temp2)
                tiles.append(tsegs)
                tile_no += 1
        tiles = np.array(tiles)
        arrayName = path+'nparrs/'+imageName[:-4]+'.npy'
        np.save(arrayName, tiles)
        frame_no += 1
        print(frame_no)

In [ ]:
images = os.listdir(path+'images')
makeNumpyArrayInputs(images)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [ ]:
image = cv2.imread('../data/images/frame1.png')
frame_no = 1
print(image.shape)
windowsize_r = 192
windowsize_c = 192
tile_no = 1
tiles = []
for r in range(0,image.shape[0], windowsize_r):
    for c in range(0,image.shape[1], windowsize_c):
        window = image[r:r+windowsize_r,c:c+windowsize_c]
        tsegs = []
        for x in range(windowsize_r):
            temp = np.append(window[x], np.ones((192,1), dtype=np.uint8)*tile_no, axis=1)
            temp2 = np.append(temp, np.ones((192,1), dtype=np.uint8)*frame_no, axis=1)
            tsegs.append(temp2)
        tiles.append(tsegs)
        tile_no += 1
tiles = np.array(tiles)
print(tiles)
print(tiles.shape)